In [1]:
# input
n_topics = 5

In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
# fixes weird issue with pyLDAvis (?) warnings

import pandas as pd
import numpy as np
import pickle
import operator
import re
import gc
import gensim
# from gensim.similarities import WmdSimilarity

import pyLDAvis
import pyLDAvis.gensim

# wtf
warnings.filterwarnings("ignore", category=DeprecationWarning)

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from tw_dataset.dbmodels import *
from random import sample
import json

Switching to API Credentials #1


# OK, let's finally do some topic modelling

In [3]:
TM_MODELS_PATH = '/media/pablo/data/Tesis/models/old/tm_feats/'
from os.path import join

In [4]:
dictionary = gensim.corpora.Dictionary.load(join(TM_MODELS_PATH,"tweets_es.dict"))

with open(join(TM_MODELS_PATH,'./tweets_es_bow.pickle'), 'rb') as f:
    bow = pickle.load(f)

In [5]:
len(dictionary.dfs)

11238

In [5]:
from math import ceil
iters = 100
passes = 10
workers = 7
chunksize = int(ceil(len(bow)*1.0/workers))

In [6]:
model_path = join(TM_MODELS_PATH,"./tweets_es_%dtopics.lda" % n_topics) 

In [ ]:
model = gensim.models.LdaMulticore(
        corpus=bow,
        id2word=dictionary,
        num_topics=n_topics,
        iterations=iters,
        alpha=0.001,
        passes=passes,
        chunksize=chunksize,
        workers=workers
)
model.save(model_path)

In [7]:
model = gensim.models.LdaModel.load(model_path)

In [ ]:
viz = pyLDAvis.gensim.prepare(model, [v for v in bow], model.id2word)
pickle.dump(viz, open("../tweets_es_%d.viz" % n_topics,'wb'))

In [ ]:
viz = pickle.load(open("../tweets_es_%d.viz" % n_topics,'rb'))

In [ ]:
pyLDAvis.display(viz)

# Términos relevantes por tema

In [8]:
feature_names = [dictionary[i] for i in range(len(dictionary))]

In [9]:
model.show_topics(n_topics)

[(0,
  u'0.015*"dia" + 0.010*"mas" + 0.008*"via" + 0.006*"nuev" + 0.006*"buen" + 0.006*"graci" + 0.006*"hoy" + 0.005*"ser" + 0.005*"vide" + 0.005*"lleg"'),
 (1,
  u'0.007*"niembr" + 0.006*"macri" + 0.006*"dic" + 0.006*"mas" + 0.006*"si" + 0.005*"tucum" + 0.005*"ahor" + 0.005*"ser" + 0.005*"via" + 0.004*"pro"'),
 (2,
  u'0.011*"argentin" + 0.009*"ahor" + 0.008*"mas" + 0.007*"mejor" + 0.007*"part" + 0.006*"libr" + 0.006*"buen" + 0.006*"vot" + 0.005*"si" + 0.005*"gan"'),
 (3,
  u'0.009*"buen" + 0.008*"hoy" + 0.006*"ahor" + 0.006*"mas" + 0.006*"fot" + 0.006*"nuev" + 0.005*"pas" + 0.005*"air" + 0.005*"vos" + 0.005*"graci"'),
 (4,
  u'0.022*"si" + 0.017*"hac" + 0.014*"mas" + 0.009*"graci" + 0.007*"pued" + 0.006*"quier" + 0.006*"buen" + 0.005*"pas" + 0.005*"bien" + 0.005*"mal"')]

# Extracción de features LDA sobre tweets

In [10]:
tweets_lda = [model[t_bow] for t_bow in bow]

In [11]:
def rows_to_csc(rows):
    data = []
    row_ind = []
    col_ind = []
    for i, r in enumerate(rows):
        for j, d in r:
            row_ind.append(i)
            col_ind.append(j)
            data.append(d)
    return csc_matrix((data, (row_ind, col_ind)))

In [12]:
from scipy.sparse import csc, csc_matrix

In [13]:
X_tweets_lda = rows_to_csc(tweets_lda)

In [14]:
s = open_session()
all_tweets_es_ids = [t[0] for t in s.query(Tweet.id, Tweet.lang).all() if t.lang == 'es']
len(all_tweets_es_ids)    

1636480

In [15]:
X_tweets_lda_df = pd.DataFrame(index=all_tweets_es_ids, data=X_tweets_lda.todense())

X_tweets_lda_df.to_pickle(join(TM_MODELS_PATH, "./alltweets_es_lda%d.pickle" % n_topics))